In [ ]:
!pip install ipywidgets matplotlib requests


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.2 MB/s eta 0:00:00


In [1]:
import os
import requests
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# WEATHER DATA FUNCTION
def get_weather_data(location, forecast_days=5):
    url = f"https://wttr.in/{location}?format=j1"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        return {"error": str(e)}

# PARSE WEATHER QUESTION FUNCTION
def parse_weather_question(question):
    result = {"location": "", "attribute": ""}
    if "temperature" in question.lower():
        result["attribute"] = "temperature"
    elif "rain" in question.lower() or "precipitation" in question.lower():
        result["attribute"] = "precipitation"
    else:
        result["attribute"] = "general"

    words = question.split()
    for i, word in enumerate(words):
        if word.lower() == "in" and i + 1 < len(words):
            result["location"] = words[i + 1]
    return result

# GENERATE RESPONSE FUNCTION
def generate_weather_response(parsed_question, weather_data):
    if "error" in weather_data:
        return f"Error: {weather_data['error']}"

    location = parsed_question["location"]
    attribute = parsed_question["attribute"]
    current = weather_data.get("current_condition", [{}])[0]

    if attribute == "temperature":
        return f"🌡️ The current temperature in {location} is {current.get('temp_C')}°C."
    elif attribute == "precipitation":
        return f"🌧️ The current chance of precipitation in {location} is {current.get('precipMM')} mm."
    else:
        return f"🌤️ In {location}, it's currently {current.get('weatherDesc')[0]['value']} with a temperature of {current.get('temp_C')}°C."

# TEMPERATURE VISUALISATION FUNCTION
def create_temperature_visualisation(weather_data):
    forecast = weather_data.get("weather", [])
    dates = [day["date"] for day in forecast]
    temps = [int(day["avgtempC"]) for day in forecast]

    fig, ax = plt.subplots()
    ax.plot(dates, temps, marker='o', linestyle='-', linewidth=2)
    ax.set_title("📈 Temperature Trend")
    ax.set_ylabel("Temperature (°C)")
    ax.set_xlabel("Date")
    plt.grid(True)
    plt.show()

# PRECIPITATION VISUALISATION FUNCTION
def create_precipitation_visualisation(weather_data):
    forecast = weather_data.get("weather", [])
    dates = [day["date"] for day in forecast]
    precipitation = [float(day["hourly"][0]["precipMM"]) for day in forecast]

    fig, ax = plt.subplots()
    ax.bar(dates, precipitation, color='skyblue')
    ax.set_title("🌧️ Precipitation Forecast")
    ax.set_ylabel("Precipitation (mm)")
    ax.set_xlabel("Date")
    plt.grid(True)
    plt.show()

# INTERFACE
def launch_weatherwise_interface():
    output = widgets.Output()
    location_input = widgets.Text(value='Port Louis', description='Location:', style={'description_width': 'initial'})
    forecast_input = widgets.BoundedIntText(value=3, min=1, max=5, description='Days (1-5):', style={'description_width': 'initial'})
    question_input = widgets.Textarea(value='', placeholder='Ask a weather question...', layout=widgets.Layout(width='100%', height='60px'))
    fetch_button = widgets.Button(description='Fetch Weather', button_style='info')
    temp_button = widgets.Button(description='Show Temperature Chart', button_style='success')
    rain_button = widgets.Button(description='Show Precipitation Chart', button_style='primary')
    ai_button = widgets.Button(description='Ask AI about Weather', button_style='warning')

    data_store = {}

    def on_fetch_clicked(b):
        with output:
            clear_output()
            print("🌍 Fetching weather data...")
            loc = location_input.value
            days = forecast_input.value
            data_store["weather"] = get_weather_data(loc, days)
            if "error" in data_store["weather"]:
                print(f"❌ Error: {data_store['weather']['error']}")
            else:
                print(f"✅ Weather data for {loc} retrieved successfully!")

    def on_temp_clicked(b):
        with output:
            clear_output()
            if "weather" in data_store:
                create_temperature_visualisation(data_store["weather"])
            else:
                print("⚠️ Please fetch the weather data first.")

    def on_rain_clicked(b):
        with output:
            clear_output()
            if "weather" in data_store:
                create_precipitation_visualisation(data_store["weather"])
            else:
                print("⚠️ Please fetch the weather data first.")

    def on_ai_clicked(b):
        with output:
            clear_output()
            if "weather" not in data_store:
                print("⚠️ Please fetch the weather data first.")
                return
            question = question_input.value
            parsed = parse_weather_question(question)
            parsed["location"] = location_input.value
            response = generate_weather_response(parsed, data_store["weather"])
            print(f"🤖 AI Response:\n{response}")

    fetch_button.on_click(on_fetch_clicked)
    temp_button.on_click(on_temp_clicked)
    rain_button.on_click(on_rain_clicked)
    ai_button.on_click(on_ai_clicked)

    display(widgets.VBox([
        widgets.HTML("<h2>🌦️ <b>WeatherWise: Intelligent Weather Assistant</b></h2>"),
        location_input,
        forecast_input,
        widgets.HBox([fetch_button, temp_button, rain_button]),
        question_input,
        ai_button,
        output
    ]))

# launch the app
launch_weatherwise_interface()